In [16]:
import pandas as pd
import os
import numpy as np

Definições iniciais

In [17]:
#UF
ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 
       'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

In [18]:
rename = {'comum_creche': 'etapa_ensino_creche_comum',
'comum_pre': 'etapa_ensino_pre_escola_comum',
'comum_fund_ai': 'etapa_ensino_fundamental_anos_iniciais_comum',
'comum_fund_af': 'etapa_ensino_fundamental_anos_finais_comum',
'comum_medio_medio': 'etapa_ensino_medio_comum',
'comum_medio_integrado': 'etapa_ensino_medio_integrado_comum',
'comum_medio_normal': 'etapa_ensino_medio_normal_comum',
'comum_prof': 'etapa_ensino_profissional_comum',
'comum_eja_fund': 'etapa_ensino_eja_fundamental_comum',
'comum_eja_medio': 'etapa_ensino_eja_medio_comum',
'comum_eja_prof': 'etapa_ensino_eja_profissional_comum',
'especial_exclusiva_creche': 'etapa_ensino_creche_especial_exclusiva',
'especial_exclusiva_pre': 'etapa_ensino_pre_escola_especial_exclusiva',
'especial_exclusiva_fund_ai': 'etapa_ensino_fundamental_anos_iniciais_especial_exclusiva',
'especial_exclusiva_fund_af': 'etapa_ensino_fundamental_anos_finais_especial_exclusiva',
'especial_exclusiva_medio_medio': 'etapa_ensino_medio_especial_exclusiva',
'especial_exclusiva_medio_integr': 'etapa_ensino_medio_integrado_especial_exclusiva',
'especial_exclusiva_medio_normal': 'etapa_ensino_medio_normal_especial_exclusiva',
'especial_exclusiva_prof': 'etapa_ensino_profissional_especial_exclusiva',
'especial_exclusiva_eja_fund': 'etapa_ensino_eja_fundamental_especial_exclusiva',
'especial_exclusiva_eja_medio': 'etapa_ensino_eja_medio_especial_exclusiva'}

In [19]:
#variáveis do dicionário que não vão ser adicionadas agora
variable_exclude = ['etapa_ensino_creche_comum',
'etapa_ensino_pre_escola_comum',
'etapa_ensino_fundamental_anos_iniciais_comum',
'etapa_ensino_fundamental_anos_finais_comum',
'etapa_ensino_medio_comum',
'etapa_ensino_medio_integrado_comum',
'etapa_ensino_medio_normal_comum',
'etapa_ensino_profissional_comum',
'etapa_ensino_eja_fundamental_comum',
'etapa_ensino_eja_medio_comum',
'etapa_ensino_eja_profissional_comum',
'etapa_ensino_creche_especial_exclusiva',
'etapa_ensino_pre_escola_especial_exclusiva',
'etapa_ensino_fundamental_anos_iniciais_especial_exclusiva',
'etapa_ensino_fundamental_anos_finais_especial_exclusiva',
'etapa_ensino_medio_especial_exclusiva',
'etapa_ensino_medio_integrado_especial_exclusiva',
'etapa_ensino_medio_normal_especial_exclusiva',
'etapa_ensino_profissional_especial_exclusiva',
'etapa_ensino_eja_fundamental_especial_exclusiva',
'etapa_ensino_eja_medio_especial_exclusiva']

In [20]:
#arquitetura: renomear e ordernar
arquitetura = pd.read_csv(r'C:\Users\ingri\OneDrive - 033ms\workspace\br_inep_censo_escolar\extra\architecture\escola\arquitetura.csv', nrows=373, na_filter = False)
new_name_variable = arquitetura.name.tolist()
order = new_name_variable[2:]

In [21]:
#função de path
def path_out ():
    partic = 'output\escola'
    partic_aux = "\\ano={}\\sigla_uf={}"
    partic = os.path.abspath(os.path.join('..', partic))
    if (not os.path.isdir(partic)):
        os.mkdir(partic)
    path= os.path.join(partic + partic_aux)
    return path

In [22]:
#path dos arquivos da base antiga
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
path_option = []
for j in years:
    var = 'input/old_base'
    data = 'base_old_' + str(j) + '.csv'
    path = os.path.abspath(os.path.join('..', var))
    path_aux = os.path.join(path, data)
    path_option.append(path_aux)

In [48]:
#path dos arquivos da base nova
path_new_base = []
for i in years:
    path_uf = {}
    for j in ufs:
        path_uf[j] = path_out().format(i, j) + '\escola.csv'
    path_new_base.append(path_uf)

In [47]:
#path dos arquivos da base a serem add colunas
years_add_column = [2007, 2008, 2021]
path_base_add = []
for i in years_add_column:
    path_uf_add = {}
    for j in ufs:
        path_uf_add[j] = path_out().format(i, j) + '\escola.csv'
    path_base_add.append(path_uf_add)

Tratamento

In [ ]:
#merge
base_list = []
for i in range(len(years)):
    count = 0
    base_dict_uf = {}
    for j in ufs:
        count = count + 1
        print(count)
        old_base = pd.read_csv(path_option[i], na_filter = False, engine = 'python', dtype = 'string')
        base_dict_uf[j] = pd.read_csv(path_new_base[i][j], na_filter = False, engine = 'python', dtype = 'string')
        base_dict_uf[j] = old_base.merge(base_dict_uf[j], how = 'right', on = ['id_escola']).drop(['ano'], axis = 1)
    base_list.append(base_dict_uf)

In [50]:
#incluir colunas vazias nos anos de 2007, 2008, 2009 e 2010
base_list_add = []
for i in range(len(years_add_column)):
    count = 0
    base_dict_uf_add = {}
    for uf in ufs:
        base_dict_uf_add[uf] = pd.read_csv(path_base_add[i][uf], na_filter = False, engine = 'python', dtype = 'string')
        aux = base_dict_uf_add[uf]
        for j in variable_exclude:
            count = count + 1
            print(count)
            aux[j] = ''
        base_dict_uf_add[uf] =  aux[order]
    base_list_add.append(base_dict_uf_add)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Output

In [55]:
for i in range(len(years)):
    mg_a = base_list[i]
    for uf in ufs:
        path_i = path_out().format(years[i], uf)
        if (not os.path.isdir(path_i)):
            os.makedirs(path_i)
        mg_auf = mg_a[uf]
        mg_auf.rename(columns = rename, inplace = True)
        mg_auf = mg_auf[order]
        mg_auf.to_csv(path_i + '\escola.csv', index = False)
        del path_i

In [54]:
for i in range(len(years_add_column)):
    mg_a = base_list_add[i]
    for uf in ufs:
        path_i = path_out().format(years_add_column[i], uf)
        if (not os.path.isdir(path_i)):
            os.makedirs(path_i)
        mg_auf = mg_a[uf]
        mg_auf = mg_auf[order]
        mg_auf.to_csv(path_i + '\escola.csv', index = False)
        del path_i